In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

## Resources

In [2]:
# Today price: https://selectra.es/energia/info/que-es/precio-kwh
# Tomorrow price: https://selectra.es/energia/info/que-es/precio-kwh#precio-luz-manana
# MoM evolution: https://selectra.es/energia/info/que-es/precio-kwh#precio-luz-hoy

# Overall metrics: https://tarifaluzhora.es/
# Electro devices per year: https://tarifaluzhora.es/info/consumo-mensual-luz#como-reparte-consumo

# Consumption per device per year: https://comparadorluz.com/faq/consumo-medio-casa#consumo-luz-electrodomesticos
# Consumption per device: https://comparadorluz.com/faq/consumo-medio-casa#consumo-luz-electrodomesticos

## Scraping today's prices

In [3]:
url = 'https://tarifaluzhora.es/'

headers= {'Accept-Encoding':'gzip, deflate',
          'Accept-Language':'en-US,en;q=0.9',
          'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}

response = requests.get(url, headers = headers)
response.status_code

soup = BeautifulSoup(response.content)
#print(soup.prettify())

average_today_price = soup.find('div',attrs={'class':'inner_block gauge_day'})
average_today_price = average_today_price.span.string[:6]

cheapest_today_time = soup.find('div',attrs={'class':'inner_block gauge_low'})
cheapest_today_time = cheapest_today_time.span.string.replace('h','')

expensive_today_time = soup.find('div',attrs={'class':'inner_block gauge_hight'})
expensive_today_time = expensive_today_time.span.string.replace('h','')

lowest_today_price = soup.find('span',attrs={'class':'sub_text green'})
lowest_today_price= lowest_today_price.get_text().replace('\t','').replace('\n','')[:7]

highest_today_price = soup.find('span',attrs={'class':'sub_text red'})
highest_today_price= highest_today_price.get_text().replace('\t','').replace('\n','')[:7]

print(f'Average price of today is: {average_today_price}€ per kwh')
print(f'Lowest price of today is: {lowest_today_price}€ per kwh')
print(f'Highest price of today is: {highest_today_price}€ per kwh')
print(f'Cheapest time of today was between {cheapest_today_time} hs')
print(f'Expensive time of today was between {expensive_today_time} hs')

hour_range = soup.find_all('span',attrs={'itemprop':'description'})
hour_price = soup.find_all('span',attrs={'itemprop':'price'})

hours = []

for i in hour_range:
    hours.append(i.string.replace(':','').replace('h',''))
    
prices = []

for i in hour_price:
    prices.append(i.string.replace('€/kW','').replace('h','').strip())

Average price of today is: 0.2335€ per kwh
Lowest price of today is: 0.14016€ per kwh
Highest price of today is: 0.28201€ per kwh
Cheapest time of today was between 17 - 18 hs
Expensive time of today was between 21 - 22 hs


## Get today prices from https://tarifaluzhora.es/

In [4]:

price_per_hour = pd.DataFrame()  
price_per_hour['Hours'] = hours
price_per_hour['Prices'] = prices
price_per_hour['Prices'] = price_per_hour['Prices'].astype(float)
price_per_hour['Avg Today'] = price_per_hour['Prices'].mean()
price_per_hour['Deviation'] = price_per_hour['Prices'].mean() - price_per_hour['Prices']

def price_flag(x):
    if x<0:
        return 'Expensive'
    elif x>0: 
        return 'Cheaper'
    else:
        return 'Regular'

price_per_hour['Price Flag'] = price_per_hour['Deviation'].apply(price_flag)

def light_consumption(x):
    return x*0.117

def kitchen_consumption(x):
    return x*0.093

def hot_water_consumption(x):
    return x*0.075

def heat_consumption(x):
    return x*0.074

def refrigeration_consumption(x):
    return x*0.023

def standby_consumption(x):
    return x*0.066

def home_appliances_consumption(x):
    return x*0.552

price_per_hour['Home Appliances'] = price_per_hour['Prices'].apply(home_appliances_consumption)
price_per_hour['Light'] = price_per_hour['Prices'].apply(light_consumption)
price_per_hour['Kitchen'] = price_per_hour['Prices'].apply(kitchen_consumption)
price_per_hour['Hot water'] = price_per_hour['Prices'].apply(hot_water_consumption)
price_per_hour['Heat'] = price_per_hour['Prices'].apply(heat_consumption)
price_per_hour['Refrigeration'] = price_per_hour['Prices'].apply(refrigeration_consumption)
price_per_hour['Stand By'] = price_per_hour['Prices'].apply(standby_consumption)

price_per_hour

,Hours,Prices,Avg Today,Deviation,Price Flag,Home Appliances,Light,Kitchen,Hot water,Heat,Refrigeration,Stand By
0,00 - 01,0.25791,0.233525,-0.024385,Expensive,0.142366,0.030175,0.023986,0.019343,0.019085,0.005932,0.017022
1,01 - 02,0.25588,0.233525,-0.022355,Expensive,0.141246,0.029938,0.023797,0.019191,0.018935,0.005885,0.016888
2,02 - 03,0.25739,0.233525,-0.023865,Expensive,0.142079,0.030115,0.023937,0.019304,0.019047,0.005920,0.016988
3,03 - 04,0.25658,0.233525,-0.023055,Expensive,0.141632,0.030020,0.023862,0.019243,0.018987,0.005901,0.016934
4,04 - 05,0.26422,0.233525,-0.030695,Expensive,0.145849,0.030914,0.024572,0.019817,0.019552,0.006077,0.017439
5,05 - 06,0.26951,0.233525,-0.035985,Expensive,0.148770,0.031533,0.025064,0.020213,0.019944,0.006199,0.017788
6,06 - 07,0.26156,0.233525,-0.028035,Expensive,0.144381,0.030603,0.024325,0.019617,0.019355,0.006016,0.017263
7,07 - 08,0.26205,0.233525,-0.028525,Expensive,0.144652,0.030660,0.024371,0.019654,0.019392,0.006027,0.017295
8,08 - 09,0.24970,0.233525,-0.016175,Expensive,0.137834,0.029215,0.023222,0.018728,0.018478,0.005743,0.016480
9,09 - 10,0.23995,0.233525,-0.006425,Expensive,0.132452,0.028074,0.022315,0.017996,0.017756,0.005519,0.015837


## Scraping tomorrow's prices

In [5]:
import datetime
now = datetime.datetime.now().strftime("%I:%M %p")
prices_tomorrow_time = '8:15 PM'

def scrape_tomorrow():
    #Backup comment as a reminder
    print (f'Its {now} right now. Tomorrow prices for Spain can only be checked after 8:15 PM GMT+2. Come back later if you get zero values.')

    url = 'https://tarifaluzhora.es/info/precio-kwh-manana'

    headers= {'Accept-Encoding':'gzip, deflate',
              'Accept-Language':'en-US,en;q=0.9',
              'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}

    response = requests.get(url, headers = headers)
    response.status_code
    soup = BeautifulSoup(response.content)


    tomorrow_prices_df = pd.read_html(url)[0]
    tomorrow_prices_df['Hora'] = tomorrow_prices_df['Hora'].str.replace("h","")
    tomorrow_prices_df['Península,  Baleares y Canarias'] = tomorrow_prices_df['Península,  Baleares y Canarias'].str.replace('€/kWh','')
    tomorrow_prices_df['Ceuta y Melilla'] = tomorrow_prices_df['Ceuta y Melilla'].str.replace("€/kWh","")
    tomorrow_prices_df[['Península,  Baleares y Canarias','Ceuta y Melilla']] = tomorrow_prices_df[['Península,  Baleares y Canarias','Ceuta y Melilla']].astype(float)
    tomorrow_prices_df = tomorrow_prices_df.rename(columns={'Hora':'Hour','Península,  Baleares y Canarias': 'Península, Baleares y Canarias (in €/kWh)', 'Ceuta y Melilla': 'Ceuta y Melilla (in €/kWh)'})
    tomorrow_prices_df['Avg Today - Tier 1'] = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean()
    tomorrow_prices_df['Deviation - Tier 1'] = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean() - tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)']
    tomorrow_prices_df['Avg Today - Tier 2'] = tomorrow_prices_df['Ceuta y Melilla (in €/kWh)'].mean()
    tomorrow_prices_df['Deviation - Tier 2'] = tomorrow_prices_df['Ceuta y Melilla (in €/kWh)'].mean() - tomorrow_prices_df['Ceuta y Melilla (in €/kWh)'].mean()
    tomorrow_prices_df['Price Flag - Tier 1'] = tomorrow_prices_df['Deviation - Tier 1'].apply(price_flag)
    tomorrow_prices_df['Price Flag - Tier 2'] = tomorrow_prices_df['Deviation - Tier 2'].apply(price_flag)

    #display(tomorrow_prices_df.describe())
    return tomorrow_prices_df

scrape_tomorrow()

Its 01:18 PM right now. Tomorrow prices for Spain can only be checked after 8:15 PM GMT+2. Come back later if you get zero values.


,Hour,"Península, Baleares y Canarias (in €/kWh)",Ceuta y Melilla (in €/kWh),Avg Today - Tier 1,Deviation - Tier 1,Avg Today - Tier 2,Deviation - Tier 2,Price Flag - Tier 1,Price Flag - Tier 2
0,00,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular
1,01,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular
2,02,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular
3,03,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular
4,04,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular
5,05,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular
6,06,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular
7,07,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular
8,08,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular
9,09,0.0,0.0,0.0,0.0,0.0,0.0,Regular,Regular


In [6]:
# Getting tomorrow prices after 20:15 GMT from https://tarifaluzhora.es/info/precio-kwh-manana
# The function is not working

if now >prices_tomorrow_time:
    scrape_tomorrow() 
else :
    print (f'Its {now} right now. Tomorrow prices for Spain can only be checked after 8:15 PM GMT+2. Come back later.')

Its 01:18 PM right now. Tomorrow prices for Spain can only be checked after 8:15 PM GMT+2. Come back later.


## Telegram bot calculations

In [7]:
# Check multipliers by appliances an person per house https://comparadorluz.com/faq/consumo-medio-casa#consumo-luz-electrodomesticos

# We should update these variables based on user choice

#devices_list = [washing_machine, oven, house_size, heater]
#washing_machine = input()
#oven = input()
#house_size = input()
#heater = input()


def light_consumption(x):
    return x*0.117

def kitchen_consumption(x):
    return x*0.093

def hot_water_consumption(x):
    return x*0.075

def heat_consumption(x):
    return x*0.074

def refrigeration_consumption(x):
    return x*0.023

def standby_consumption(x):
    return x*0.066

def home_appliances_consumption(x):
    return x*0.552